In [57]:
pip install pyrealsense2 opencv-python mediapipe

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [58]:
import pyrealsense2 as rs
import cv2
import mediapipe as mp
import numpy as np
import math

In [59]:
def get_robot_tcp():
    return [0.5, 0.2, 2]

def draw_tcp_on_image(tcp_coords, depth_intrin, color_image):
    tcp_pixel_coords = rs.rs2_project_point_to_pixel(depth_intrin, tcp_coords)

    tcp_x, tcp_y = int(tcp_pixel_coords[0]), int(tcp_pixel_coords[1])

    height, width, _ = color_image.shape
    if 0 <= tcp_x < width and 0 <= tcp_y < height:
        cv2.circle(color_image, (tcp_x, tcp_y), 10, (0, 0, 255), -1)  # Red circle
        cv2.putText(color_image, "TCP", (tcp_x + 15, tcp_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

# Euclidean distance between two 3D points
def calculate_distance(point1, point2):
    return math.sqrt((point1[0] - point2[0]) ** 2 +
                     (point1[1] - point2[1]) ** 2 +
                     (point1[2] - point2[2]) ** 2)

In [60]:
# Initialize MediaPipe Pose detection
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Configure RealSense pipeline for both color and depth streams
pipeline = rs.pipeline()
config = rs.config()

# Enable both color and depth streams
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Start the RealSense camera stream
pipeline.start(config)

# Align depth frame to color frame
align_to = rs.stream.color
align = rs.align(align_to)

# Safety distance threshold (in meters)
SAFETY_DISTANCE = 0.5

# Set up a resizable window
cv2.namedWindow('Pose Detection', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Pose Detection', 1280, 720)


try:
    while True:
        # Get frames from the RealSense camera
        frames = pipeline.wait_for_frames()

        # Align depth frame to color frame
        aligned_frames = align.process(frames)

        # Get aligned frames
        depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        # Convert RealSense frame to numpy array
        color_image = np.asanyarray(color_frame.get_data())
        depth_image = np.asanyarray(depth_frame.get_data())

        rgb_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)

        # Perform pose estimation using MediaPipe
        results = pose.process(rgb_image)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                color_image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=4, circle_radius=5),
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=4, circle_radius=5),
            )

            tcp_coords = get_robot_tcp()

            draw_tcp_on_image(tcp_coords, depth_frame.profile.as_video_stream_profile().intrinsics, color_image)

            too_close = False

            depth_intrin = depth_frame.profile.as_video_stream_profile().intrinsics
            height, width, _ = color_image.shape

            # Check distance for each human landmark
            for id, landmark in enumerate(results.pose_landmarks.landmark):
                # Convert to pixel coordinates
                cx = int(landmark.x * width)
                cy = int(landmark.y * height)

                # Ensure the coordinates are within the bounds of the image
                if cx >= 0 and cx < width and cy >= 0 and cy < height:
                    depth_value = depth_frame.get_distance(cx, cy)

                    human_coords = rs.rs2_deproject_pixel_to_point(depth_intrin, [cx, cy], depth_value)

                    distance = calculate_distance(tcp_coords, human_coords)

                    # Display the distance for each landmark (f"ID:{id} D:{distance:.2f}m")
                    cv2.putText(color_image, f"D:{distance:.2f}m",
                                (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 1, cv2.LINE_AA)


                    if distance < SAFETY_DISTANCE:
                        too_close = True
                else:
                    continue

            if too_close:
                print("Robot too close to human! Slowing down...")

        cv2.imshow('Pose Detection', color_image)

        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Stop the camera stream
    pipeline.stop()
    cv2.destroyAllWindows()


I0000 00:00:1727100451.619766    5832 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1727100451.622302   65580 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) UHD Graphics 630 (CFL GT2)
W0000 00:00:1727100451.729611   65577 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727100451.771003   65570 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing down...
Robot too close to human! Slowing 